# 🚀 SectorIQ: The Enterprise Knowledge Accelerator
### 🎯 Track: Enterprise Agents

---

## 1. The Problem 
**"Onboarding is slow and expensive."**
In high-paced industries (Consulting, VC, Tech), junior employees spend almost**40% of their first week** manually searching for information to understand a new domain. This process is inefficient, unstructured, and unmeasurable.

## 2. The Solution 
**SectorIQ** is an autonomous multi-agent system designed to accelerate the knowledge acquisition workflow. 
It transforms a simple keyword (e.g., "Generative AI Supply Chain") into a comprehensive training module.

**The Workflow:**
1.  **Researcher Agent:** Uses **Tavily Search API** to aggregate real-time data from trusted sources.
2.  **Analyst Agent:** Synthesizes data into a McKinsey-style **MECE report** using **Gemini-2.5-flash**.
3.  **Tutor Agent:** Generates a quiz to verify user comprehension, closing the **Learning Loop**.

## 3. Technical Architecture 
This project demonstrates **5 Key Concepts** from the course:

1.  **Multi-Agent System (Sequential):** - Implemented using **LangGraph**. The workflow chains a `Researcher` (Information Gathering) -> `Analyst` (Synthesis) -> `Tutor` (Assessment) to perform complex tasks.
2.  **Tools (External Search):** - The agents utilize **Tavily Search API** to access real-time market data, grounding the LLM's output in reality to prevent hallucinations.
3.  **State Management:** - Utilized a typed `AgentState` to maintain context and pass structured data (raw content, reports, quizzes) seamlessly between agents.
4.  **Agent Evaluation:** - Included a self-reflection loop where an LLM evaluator scores the quality of the generated report (see Step 8).
5.  **Agent Deployment:** - Fully deployed as a user-friendly Web App using **Streamlit Cloud**.
    - **👉 Try the Live App:** https://sectoriq-g8qltro6uyekbwctpk5c6r.streamlit.app/
    - Please notice: User need to use their own API Keys to run this web app.

In [11]:
# config environment
!pip install -U -q google-generativeai langchain-google-genai langgraph tavily-python langchain fpdf pandas

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [12]:
#---TEST libraries---
try:
    import langchain
    import langgraph
    import langchain_google_genai
    import tavily
    import fpdf
    print("✅ Success! You can ignore warnings.")
except ImportError as e:
    print(f"❌ NEED TO DEBUG: {e}")

✅ Success! You can ignore warnings.


In [13]:
import os
import smtplib
import base64
from email.message import EmailMessage
from kaggle_secrets import UserSecretsClient
from typing import TypedDict, List
from IPython.display import display, Markdown, HTML

# --- 1. Load Secrets ---
user_secrets = UserSecretsClient()

try:
    os.environ["GOOGLE_API_KEY"] = user_secrets.get_secret("GOOGLE_API_KEY")
    os.environ["TAVILY_API_KEY"] = user_secrets.get_secret("TAVILY_API_KEY")
    print("✅ API Keys loaded successfully.")
except Exception as e:
    print(f"⚠️ Error loading keys: {e}")
    print("Please go to 'Add-ons' -> 'Secrets' and set GOOGLE_API_KEY and TAVILY_API_KEY")

# Email Keys (optional，if user doesn't set, it will not work)
EMAIL_USER = None
EMAIL_PASSWORD = None
try:
    EMAIL_USER = user_secrets.get_secret("EMAIL_USER")
    EMAIL_PASSWORD = user_secrets.get_secret("EMAIL_PASSWORD")
    if EMAIL_USER and EMAIL_PASSWORD:
        print("✅ Email Credentials loaded.")
except:
    print("ℹ️ Email credentials not found. Email feature will be disabled.")

✅ API Keys loaded successfully.
ℹ️ Email credentials not found. Email feature will be disabled.


In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, END
from tavily import TavilyClient

# --- 2. Define State ---
class AgentState(TypedDict):
    topic: str
    user_email: str             # for those who has Emial key
    raw_content: str
    reading_list: List[dict]    # reading_list (title + url)
    structured_report: str
    quiz: str
    feedback: str              # save QA review of step8
    retry_count: int           # avoid loop lock

# --- 3. Initialize Models ---
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
print("✅ Models Initialized: Using Gemini 2.5 Flash")

✅ Models Initialized: Using Gemini 2.5 Flash


In [17]:
# --- 4. Define Nodes ---

def researcher_agent(state: AgentState):
    """Agent 1: researcher"""
    print(f"🕵️ [Researcher] Deep searching for: {state['topic']}...")
    
    # use searcher tool
    try:
        # do more search for creating reading_list
        results = tavily.search(
        query=f"{state['topic']} industry analysis statistics challenges 2025", 
        search_depth="advanced", 
        max_results=7,
        include_raw_content=True 
    )
        # get more context for analyst
        context = "\n\n".join([r['content'] for r in results['results']])
        # get reading list 
        reading_list = [{"title": r['title'], "url": r['url']} for r in results['results']]
    
    except Exception as e:
        context = f"Search failed: {e}"
        reading_list = []
        
    return {"raw_content": context, "reading_list": reading_list}

def analyst_agent(state: AgentState):
    """Agent 2: Analyst - Generate McKinsey-style report"""
    feedback = state.get('feedback', '')
    retry_count = state.get('retry_count', 0)
    
    if feedback:
        print(f"🔄 [Analyst] Rewriting based on feedback (Attempt {retry_count + 1})...")
        prompt = f"""
        PREVIOUS FEEDBACK: The editor rejected your last draft because: {feedback}.
        
        Please rewrite the report for '{state['topic']}' improving on those points.
        Use the Data: {state['raw_content']}
        """
    else:
        print(f"🧠 [Analyst] Writing first draft...")
        # add CoT into prompt
        prompt = f"""
    You are a Senior Strategy Consultant at McKinsey. 
    Your goal is to write a high-quality, data-driven industry report on: '{state['topic']}'.

    Data provided:
    {state['raw_content']}

    --- INSTRUCTIONS ---
    1. First, silently analyze the data to identify the top 3 most critical trends.
    2. Filter out any irrelevant or marketing-heavy information.
    3. Structure your report using the MECE principle (Mutually Exclusive, Collectively Exhaustive).
    4. Write the report in professional Markdown.

    --- REPORT STRUCTURE ---
    # {state['topic']}

    ## 1. Executive Summary
    (Synthesize the core insight in 3-4 powerful sentences)

    ## 2. Market Landscape
    (Use bullet points with specific data/numbers from the search results)

    ## 3. Strategic Risks & Opportunities
    (Analyze the "So What?")
                    """
        
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"structured_report": response.content, "retry_count": retry_count + 1}

def qa_gatekeeper(state: AgentState):
    print("⚖️ [QA] Reviewing draft...")
    # evaluate simply
    eval_prompt = f"Rate this report (1-10) and give 1 sentence critique. Format: Score | Critique. Report: {state['structured_report'][:2000]}"
    response = llm.invoke([HumanMessage(content=eval_prompt)])
    
    try:
        score_str, critique = response.content.split("|")
        score = int(score_str.strip())
    except:
        score = 5 # if can't analyse, give low degree metric
        critique = "Structure is unclear."
        
    return {"qa_score": score, "feedback": critique} 
        
def tutor_agent(state: AgentState):
    """Agent 3: Tutor - Generate quiz for user"""
    print(f"🎓 [Tutor] Generating quiz...")
    
    prompt = f"""
    Based on the report below, create 3 multiple-choice questions (MCQs) to test a new employee's understanding.
    Format the output cleanly with the question, options, and the correct answer hidden at the end.
    
    Report:
    {state['structured_report']}
    """
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"quiz": response.content}

def should_continue(state: AgentState):
    score = state.get("qa_score", 0)
    retry = state.get("retry_count", 0)
    
    # set pass rule
    if score >= 8 or retry >= 2:
        return "tutor"  
    else:
        return "analyst"

In [18]:
# --- 5. Tool functions ---
from fpdf import FPDF

# Tool 1: Generate PDF 
def generate_pdf_bytes(topic, report, quiz, reading_list):
    """generate PDF, and binary"""
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    
    # Title
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, f"SectorIQ: {topic}", ln=True, align='C')
    pdf.ln(10)
    
    # Report Body
    pdf.set_font("Arial", size=11)
    # Avoid Unicode error
    clean_report = report.encode('latin-1', 'replace').decode('latin-1')
    pdf.multi_cell(0, 6, clean_report)
    pdf.ln(10)
    
    # Reading List Section
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(0, 10, "Curated Reading List", ln=True)
    pdf.set_font("Arial", size=10)
    for item in reading_list:
        title = item['title'].encode('latin-1', 'replace').decode('latin-1')
        url = item['url'].encode('latin-1', 'replace').decode('latin-1')
        pdf.multi_cell(0, 6, f"- {title}\n  {url}")
        pdf.ln(2)
        
    # Quiz Section
    pdf.ln(5)
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(0, 10, "Assessment Quiz", ln=True)
    pdf.set_font("Arial", size=11)
    clean_quiz = quiz.encode('latin-1', 'replace').decode('latin-1')
    pdf.multi_cell(0, 6, clean_quiz)
    
    # Binary
    return pdf.output(dest="S").encode("latin-1")

# Tool 2: Send Email
def send_email_with_pdf(to_email, topic, pdf_bytes):
    """send email with attachment"""
    if not EMAIL_USER or not EMAIL_PASSWORD or not to_email:
        return "⚠️ Email credentials missing or no email provided. Skipping."
    
    msg = EmailMessage()
    msg['Subject'] = f"SectorIQ Training Kit: {topic}"
    msg['From'] = EMAIL_USER
    msg['To'] = to_email
    msg.set_content(f"""
    Hello,
    
    Here is your AI-generated onboarding package for '{topic}'.
    It includes an executive briefing, a curated reading list, and an assessment quiz.
    
    Powered by SectorIQ Enterprise Agent.
    """)
    
    # add attachment
    msg.add_attachment(
        pdf_bytes,
        maintype='application',
        subtype='pdf',
        filename=f"SectorIQ_{topic.replace(' ', '_')}.pdf"
    )
    
    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(EMAIL_USER, EMAIL_PASSWORD)
            smtp.send_message(msg)
        return f"✅ Email successfully sent to {to_email}!"
    except Exception as e:
        return f"❌ Email failed: {str(e)}"

In [19]:
# --- 6. Build Graph ---
workflow = StateGraph(AgentState)

# set node
workflow.add_node("researcher", researcher_agent)
workflow.add_node("analyst", analyst_agent)
workflow.add_node("qa", qa_gatekeeper)
workflow.add_node("tutor", tutor_agent)

# set edge
workflow.set_entry_point("researcher")
workflow.add_edge("researcher", "analyst")
workflow.add_edge("analyst", "qa")
workflow.add_conditional_edges(
    "qa",
    should_continue,
    {
        "analyst": "analyst", 
        "tutor": "tutor"      
    }
)
workflow.add_edge("tutor", END)

# compile graph
app = workflow.compile()
print("✅ SectorIQ Workflow is ready!")

✅ SectorIQ Workflow is ready!


In [20]:
# --- for users who want to donwload output, create a download link ---
def create_download_link(pdf_bytes, filename):
    b64 = base64.b64encode(pdf_bytes).decode()
    return f'<a href="data:application/pdf;base64,{b64}" download="{filename}" style="background-color:#28a745;color:white;padding:10px 20px;text-decoration:none;border-radius:5px;font-weight:bold;">📥 Download PDF Report</a>'
    
# --- 7. Run & Display ---

def run_sector_iq_enterprise(topic, email_address=""):
    
    display(Markdown(f"# 🚀 Starting Session: **{topic}**"))
    display(Markdown("---"))
    
    # run agent
    inputs = {"topic": topic, "user_email": email_address}
    result = app.invoke(inputs)
    
    # display result
    display(Markdown("## 📊 Insight Report"))
    display(Markdown(result['structured_report']))
    
    display(Markdown("## 📚 Curated Reading List"))
    for item in result['reading_list']:
        print(f"🔗 {item['title']} - {item['url']}")
        
    display(Markdown("---"))
    display(Markdown("## 📝 Quiz"))
    display(Markdown(result['quiz']))

    # generate PDF
    print("\n📄 Generating PDF Document...")
    pdf_bytes = generate_pdf_bytes(
        topic, 
        result['structured_report'], 
        result['quiz'], 
        result['reading_list']
    )
    
    # Define the file_name
    filename = f"SectorIQ_{topic.replace(' ', '_')}.pdf"

    # save locally (Option 1)
    with open(filename, "wb") as f:
        f.write(pdf_bytes)
    print(f"✅ PDF saved locally: {filename}")

    # show download button (Option 2)
    display(Markdown("---"))
    display(Markdown("### 💾 Export Results"))
    display(HTML(create_download_link(pdf_bytes, filename)))
    
    # send email (Option 3)
    if email_address:
        print(f"\n📧 Sending to {email_address}...")
        
        status = send_email_with_pdf(email_address, topic, pdf_bytes)
        print(status)
    else:
        print("\nℹ️ No email provided, skipping delivery.")

    return result


# ==========================================
# 👇 INPUT HERE 
# ==========================================
target_topic = "how to become an AI product manager" 
user_email = "" # if you want to use this function, you should set your email key in Cell2

# Run!
final_output = run_sector_iq_enterprise(target_topic, user_email)

# 🚀 Starting Session: **how to become an AI product manager**

---

🕵️ [Researcher] Deep searching for: how to become an AI product manager...
🧠 [Analyst] Writing first draft...
⚖️ [QA] Reviewing draft...
🔄 [Analyst] Rewriting based on feedback (Attempt 2)...
⚖️ [QA] Reviewing draft...
🎓 [Tutor] Generating quiz...


## 📊 Insight Report

## How to Become an AI Product Manager (in 2025): Your Roadmap to a High-Growth Role

### Navigate the Evolving Landscape and Build Your Path to Success at Top Companies like Nvidia, Meta, Microsoft, and OpenAI.

**By Aakash Gupta and Dr. Nancy Li**
*Updated August 14, 2025*

The field of AI Product Management is not just burgeoning; it's rapidly shaping industries in real-time. With professionals in this domain earning, on average, between $150,000 and $250,000 annually, it represents one of the hottest and fastest-growing career paths in product management. This guide provides a systematic roadmap, drawing on insights from industry leaders and experts, to help you break into this transformative role.

### Understanding the AI Product Manager Role

At its core, an AI Product Manager is a Product Manager specializing in building AI features into products or developing entirely new AI products. They act as a crucial bridge between users, technology, and business objectives, ensuring user needs are understood by engineers and technical challenges are communicated effectively to stakeholders.

However, the AI PM role comes with unique complexities:

*   **Navigate Data Complexities:** Ensuring datasets are clean, unbiased, and representative is a frequent bottleneck. Even minor data flaws can significantly derail an AI project.
*   **Manage Model Lifecycles:** Unlike static features, AI models demand continuous training, refinement, and monitoring to maintain effectiveness and relevance.
*   **Balance Ethics and Innovation:** AI PMs face the dual challenge of driving cutting-edge advancements while addressing critical issues like data bias, privacy, and model transparency. They must identify risks such as bias, fairness, hallucination, privacy, and explainability, ensuring alignment with responsible AI practices and compliance requirements.

McKinsey identifies four critical skills for successfully developing AI products, including **Low-Code Prototyping** and the ability to **Act as a Translator Across Teams**. This latter skill is paramount, involving bridging understanding between AI practitioners (data scientists, ML engineers) and non-technical stakeholders (executives, customers), and communicating tradeoffs in model choices, explainability, and ethical considerations clearly and confidently.

The industry also grapples with significant challenges that AI PMs must address. A global **shortage of AI talent and expertise** means companies struggle to find and retain individuals with the right knowledge. Furthermore, **data scarcity** is a pervasive issue; a recent IBM Institute survey found 42% of business leaders worry they lack sufficient proprietary data to effectively train or customize AI models. Developing a robust data strategy, treating data as a strategic asset, and breaking down silos are therefore critical responsibilities for an AI PM.

### The Path Less Traveled: Gaining Hands-On Experience

Many aspiring AI Product Managers wonder how to transition into the field, especially "with no experience." Our extensive interviews with over 20 AI hiring managers at top companies like Google, Amazon, and Meta revealed an overwhelmingly clear message: **practical experience in building and delivering AI/ML products is paramount.**

While foundational knowledge gained through courses is valuable, it's often seen as a prerequisite or complementary step. Hiring managers consistently emphasize that the most compelling candidates are those who have demonstrated the ability to bring products from conception ("0 to 1") and deeply understand the technical challenges inherent in implementing AI solutions.

Consequently, **one of the most impactful and direct paths** to impressing AI hiring managers is to launch an AI product that achieves genuine user adoption. This hands-on approach offers tangible evidence of your capabilities, far beyond what a resume or certification alone can convey.

### Your Step-by-Step Guide to Launching an AI Product (and Beyond)

To effectively break into AI Product Management, consider a proactive approach focused on building. As GeekyBaller, a popular AI PM educator, advises: "The best way to successfully break into AI Product Management is by launching an AI product."

Here's a roadmap to guide your journey:

1.  **Start Small and Iterate Quickly:** "In the early stages of your ML project, aim for speed. Choose the solution that validates/rejects your hypotheses the fastest." This iterative approach allows for rapid learning and adaptation in a fast-evolving field.
2.  **Focus on Real User Adoption:** The goal isn't just to build, but to build something people use. This demonstrates your ability to identify market needs, execute on a solution, and drive product growth.
3.  **Consider Monetization Strategies:** As you develop, think about the business aspect. Can your AI insights justify a premium pricing model? Does your AI offer a competitive advantage that attracts more users? Answering these questions solidifies the product’s market position.
4.  **Plan for Scaling and Generalization (Step #12 in a broader guide):** Once your AI product gains traction, scaling becomes the next challenge. Expanding AI capabilities must be done without compromising performance or fairness. Key considerations include:
    *   **Expanding AI applications:** Can the same model be adapted for different use cases?
    *   **Handling increased data volume:** Will more data improve model accuracy, or does it introduce new biases?
    *   **Ensuring fairness and bias mitigation:** Are AI outputs equitable across all demographics?
    For example, a stress detection AI could be scaled to include additional factors like speech patterns or patient movement analysis to improve accuracy.

### Cultivating Essential AI PM Skills

Beyond launching your own product, continuous skill development is crucial:

*   **Develop a Robust Data Strategy:** Treat data as a strategic asset. Break down silos by creating centralized data lakes or cloud warehouses that aggregate information. Implement data integration pipelines to continuously ingest and normalize data, supported by proper data cataloging, metadata tagging, and access control.
*   **Act as a Translator:** Hone your ability to bridge understanding between highly technical AI teams and non-technical business stakeholders. This involves clearly communicating complex concepts, trade-offs, and ethical considerations.
*   **Seek Continuous Learning:** AI isn’t static—it’s an ever-evolving field. Staying relevant means keeping up with the latest advancements. Follow technical blogs like the Netflix Tech Blog and LinkedIn Engineering Blog to learn how industry leaders are solving problems. Explore cutting-edge tools, attend industry events, and continuously expand your expertise.
*   **Focus on the Journey, Not Just the Role:** Embrace iterative learning and be open to new methodologies. The path to becoming an AI PM is a dynamic one, requiring adaptability and a growth mindset.

### The Future of Product Management: Adapting to AI

The traditional product management playbook is undergoing a profound transformation. While core principles endure, AI-enabled PMs are increasingly leveraging tools like ChatGPT and Lovable to significantly enhance their productivity, often achieving 2-3x efficiency gains. By applying AI across everything from user research to prototype development, they are creating a notable productivity gap that demands attention.

The pervasive influence of AI suggests a fundamental shift: product managers will increasingly need to integrate AI understanding and tools into their practice to remain competitive and effective. Adapting to this AI-driven evolution will be key to long-term career relevance in product management. This isn't about the "death" of traditional PM, but its powerful evolution.

### Conclusion

Becoming an AI Product Manager in 2025 is an ambitious but highly rewarding goal. While the challenge of gaining experience in a specialized field is real, the most effective strategy involves demonstrating your capabilities through hands-on product creation. By understanding the unique demands of the role, proactively building and launching an AI product, and committing to continuous learning, you can carve out a successful path in this exciting and indispensable domain. Grab a notebook, hit play, and turn AI disruption into your advantage.

...

Here are 3 multiple-choice questions based on the report:

---

**Question 1:**
What is one of the *unique complexities* that an AI Product Manager must navigate, as explicitly mentioned in the report?
A) Developing traditional marketing strategies for consumer goods.
B) Ensuring datasets are clean, unbiased, and representative, as even minor flaws can significantly derail an AI project.
C) Managing the supply chain for physical product components.
D) Facilitating international trade agreements for technology exports.

<details>
<summary>Correct Answer</summary>
B) Ensuring datasets are clean, unbiased, and representative, as even minor flaws can significantly derail an AI project.
</details>

---

**Question 2:**
Based on insights from AI hiring managers in the report, what is considered the *most impactful and direct path* for aspiring AI Product Managers to gain experience?
A) Completing a series of advanced theoretical AI certifications.
B) Attending numerous industry conferences and networking extensively.
C) Demonstrating practical experience by building and launching an AI product with genuine user adoption.
D) Gaining experience primarily in non-AI traditional product management roles.

<details>
<summary>Correct Answer</summary>
C) Demonstrating practical experience by building and launching an AI product with genuine user adoption.
</details>

---

**Question 3:**
The report highlights two critical skills identified by McKinsey for successfully developing AI products and mentions a significant industry challenge AI PMs must address. Which option correctly lists these?
A) Skills: Sales & Marketing, Project Management; Challenge: User apathy towards new AI.
B) Skills: Low-Code Prototyping, Acting as a Translator Across Teams; Challenge: A global shortage of AI talent and expertise.
C) Skills: Financial Analysis, Legal Expertise; Challenge: Excessive availability of clean, proprietary data.
D) Skills: Graphic Design, Content Writing; Challenge: Over-reliance on traditional software development methods.

<details>
<summary>Correct Answer</summary>
B) Skills: Low-Code Prototyping, Acting as a Translator Across Teams; Challenge: A global shortage of AI talent and expertise.
</details>


📄 Generating PDF Document...
✅ PDF saved locally: SectorIQ_how_to_become_an_AI_product_manager.pdf


---

### 💾 Export Results


ℹ️ No email provided, skipping delivery.


In [21]:
import json
import pandas as pd

# --- 8. Agent Evaluation (Enterprise QA Module) ---
print("⚖️ Running Enterprise Quality Assurance (QA) Check...")

def run_qa_check(report_content, quiz_content):
    """
    Simulates a Senior Editor reviewing the work.
    Evaluates on 3 dimensions: Strategic Depth, Clarity, and Quiz Relevance.
    """
    
    # Define prompt (Rubric)
    qa_prompt = f"""
    You are a Senior Chief Editor at a top consulting firm.
    Evaluate the following 'Industry Report' and 'Employee Quiz'.
    
    Return the result in valid JSON format ONLY, with no extra text.
    The JSON keys must be:
    - "strategic_depth_score": (1-10, how insightful is the analysis?)
    - "clarity_score": (1-10, is the formatting and language professional?)
    - "quiz_relevance_score": (1-10, do the questions actually test the report content?)
    - "overall_score": (Average of the above)
    - "critique": (One sentence on what is good or bad)
    - "recommendation": (One actionable tip to improve)

    --- REPORT CONTENT ---
    {report_content[:3000]} 
    
    --- QUIZ CONTENT ---
    {quiz_content}
    """
    
    # call LLM
    try:
        response = llm.invoke([HumanMessage(content=qa_prompt)])
        # make sure the format is JSON
        json_str = response.content.replace("```json", "").replace("```", "").strip()
        metrics = json.loads(json_str)
        return metrics
    except Exception as e:
        print(f"QA Parsing Error: {e}")
        return None

# evaluation
qa_metrics = run_qa_check(final_output['structured_report'], final_output['quiz'])

# Visualizing Metrics
if qa_metrics:
    display(Markdown("### 🛡️ Quality Assurance Scorecard"))
    
    # 1. show result dynamically
    score = qa_metrics['overall_score']
    if score >= 8:
        status_color = "green"
        status_text = "✅ PASSED"
    elif score >= 6:
        status_color = "orange"
        status_text = "⚠️ NEEDS REVIEW"
    else:
        status_color = "red"
        status_text = "❌ REJECTED"
        
    display(HTML(f'<h3 style="color:{status_color};">{status_text} (Score: {score}/10)</h3>'))
    
    # 2. show details with dashboard
    df = pd.DataFrame([qa_metrics])
    # more readable
    df_display = df[['strategic_depth_score', 'clarity_score', 'quiz_relevance_score', 'critique', 'recommendation']].T
    df_display.columns = ['Feedback']
    display(df_display)
    
    # 3. brief comment
    print(f"\n💡 Editor's Note: {qa_metrics['recommendation']}")

else:
    print("⚠️ QA Check failed to generate metrics.")

⚖️ Running Enterprise Quality Assurance (QA) Check...


### 🛡️ Quality Assurance Scorecard

,Feedback
strategic_depth_score,7
clarity_score,9
quiz_relevance_score,6.7
critique,The report provides a strong foundational unde...
recommendation,Ensure the report is fully completed to delive...



💡 Editor's Note: Ensure the report is fully completed to deliver on its stated objectives and meticulously align all quiz questions with the final, comprehensive content.
